In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse
from collections import deque
import pprint
import os # 拡張子をチェックするために os.path を使います

# --- 設定 ---

# 1. 開始URL
START_URL = "https://www.musashino-u.ac.jp/"

# 2. クロール対象のドメイン
BASE_DOMAIN = "musashino-u.ac.jp"

# 3. 負荷軽減のための待機時間（秒）
WAIT_TIME = 1

# 4. クロールする最大ページ数（サイト全体をクロールすると時間がかかりすぎるため）
MAX_PAGES = 50 

# 5. 【NEW】クロール対象から除外するファイルの拡張子
#    これらに該当するURLはたどりません。
EXCLUDE_EXTENSIONS = [
    # 画像
    '.png', '.jpg', '.jpeg', '.gif', '.svg', '.bmp', '.webp',
    # ドキュメント
    '.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx',
    # 圧縮ファイル
    '.zip', '.rar', '.gz',
    # メディア
    '.mp4', '.mov', '.avi', '.mp3', '.wav',
    # その他
    '.css', '.js'
]

# ----------------

def crawl_sitemap_v2():
    """
    指定されたURLからクロールを開始し、サイトマップ（URL: title）の辞書を作成する。
    画像やPDFなどのファイルへのリンクは除外する。
    """
    
    # サイトマップを格納する辞書
    sitemap = {}
    
    # これから訪問するURLを管理するキュー
    queue = deque([START_URL])
    
    # 既に訪問した、またはキューに追加したURLを管理するセット
    visited_urls = {START_URL}
    
    # リクエスト時のヘッダー
    headers = {
        "User-Agent": "MySitemapScraper/1.0 (Contact: your-email@example.com)"
    }

    print(f"クロールを開始します... (最大{MAX_PAGES}ページ)")
    print(f"除外する拡張子: {EXCLUDE_EXTENSIONS}")

    while queue and len(sitemap) < MAX_PAGES:
        # キューからURLを1つ取り出す
        current_url = queue.popleft()

        print(f"[{len(sitemap) + 1}/{MAX_PAGES}] 処理中: {current_url}", end="")

        try:
            # --- 課題要件: 負荷軽減 ---
            time.sleep(WAIT_TIME)

            # --- 課題要件: ページにアクセス ---
            response = requests.get(current_url, headers=headers, timeout=10)
            
            if response.status_code != 200:
                print(" -> スキップ (Status Code)")
                continue
            
            # コンテンツタイプがHTML/テキストでない場合はスキップ
            # (万が一、拡張子チェックをすり抜けてもここで弾く)
            if 'text/html' not in response.headers.get('Content-Type', ''):
                print(" -> スキップ (HTMLではありません)")
                continue
                
            response.encoding = response.apparent_encoding
            
            soup = BeautifulSoup(response.text, "html.parser")

            # --- 課題要件: <title> を取得 ---
            title_tag = soup.find("title")
            title = "タイトルなし"
            if title_tag and title_tag.string:
                title = title_tag.string.strip()
            
            # --- 課題要件: 辞書型変数に格納 ---
            sitemap[current_url] = title
            print(f" -> {title}")

            # --- 課題要件: 同一ドメインの全リンクを巡る ---
            links = soup.find_all("a", href=True)

            for link in links:
                href = link.get("href")

                # 1. 相対URLを絶対URLに変換
                absolute_url = urljoin(current_url, href)
                
                # 2. URLを正規化
                parsed_url = urlparse(absolute_url)
                clean_url = parsed_url._replace(query="", fragment="").geturl()

                # 3. URLがクロール対象かチェック
                
                # (a) パスから拡張子を取得 (例: /foo/bar.PDF -> .pdf)
                path = parsed_url.path
                # os.path.splitext で拡張子を分離し、小文字に変換
                ext = os.path.splitext(path)[1].lower()

                # (b) チェックロジック
                is_valid = (
                    # 同一ドメインか？
                    BASE_DOMAIN in parsed_url.netloc and
                    # HTTP/HTTPSプロトコルか？
                    parsed_url.scheme in ["http", "https"] and
                    # まだ訪問済みリストにないか？
                    clean_url not in visited_urls and
                    # 【NEW】除外リストにある拡張子ではないか？
                    ext not in EXCLUDE_EXTENSIONS and
                    # そもそもパスが空（例: "https://.../"）の場合もOK
                    ext == '' 
                )

                if is_valid:
                    # 訪問済みリストとキューに追加
                    visited_urls.add(clean_url)
                    queue.append(clean_url)

        except requests.exceptions.RequestException as e:
            print(f" -> エラー ( {e} )")
        except Exception as e:
            print(f" -> 不明なエラー ( {e} )")

    print("\nクロールが完了しました。")
    return sitemap


# --- メイン処理 ---
if __name__ == "__main__":
    # クローラーを実行
    site_map_data = crawl_sitemap_v2()
    
    # --- 課題要件: 辞書型変数を print() で表示する ---
    print("\n--- サイトマップ（辞書型） ---")
    pprint.pprint(site_map_data)
    print(f"\n合計 {len(site_map_data)} ページを検出しました。")